In [ ]:
import os 
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchaudio

In [ ]:
dbg_log_dir = '/fs/scratch/PAS0774/Shanmukh/ControlledExp/random_seg/dbg_signals/simu_rirs_dbg_analysis/loss_functions_comparison_circular_array/noisy_reverb/diffuse/'
case_log_dir = 't60_0.2_snr_5.0dB_corrected_doa_sig_norm/DOA_MAG_WT'
t60, snr, wgt_mech = 0.2, 5.0, "MAG"
app_str=f't60_{t60}_snr_{snr}dB'

In [ ]:
#debugging 
#circular array analysis
log_dir = os.path.join(dbg_log_dir, case_log_dir)
loss_list = ["MIMO_RI", "MIMO_RI_MAG", "MIMO_RI_MAG_PD", "MIMO_RI_PD", "MIMO_RI_PD_REF"]
idx = 6
all_est_dict = torch.load(os.path.join(log_dir, f'doa_{app_str}_{idx}_tol_5.0deg_euclid_True_wgt_mech_{wgt_mech}.pt'))
for loss in loss_list:
    (est_f_doa, est_f_vals, est_utt_doa, est_frm_Acc, est_utt_Acc, metrics_list, est_mae_only_correct_frms, est_mae_only_incorrect_frms, est_mae_overall_only_vad_frms) = all_est_dict[f'{loss}']
    plt.figure()
    plt.title(f'{loss}')
    plt.plot(est_f_doa)
    plt.plot(est_utt_doa*torch.ones(len(est_f_doa)), 'r')
    plt.plot((360 - est_utt_doa)%360*torch.ones(len(est_f_doa)), 'g')
    plt.plot((120 - est_utt_doa)%360*torch.ones(len(est_f_doa)), 'b')
    plt.plot((240 - est_utt_doa)%360*torch.ones(len(est_f_doa)), 'y')



In [ ]:
#circular array analysis
log_dir = '/fs/scratch/PAS0774/Shanmukh/ControlledExp/random_seg/dbg_signals/simu_rirs_dbg_analysis/loss_functions_comparison_circular_array/noisy_reverb/diffuse/t60_1.0_snr_-5.0dB_corrected_doa_sig_norm'
loss_list = ["MIMO_RI", "MIMO_RI_MAG", "MIMO_RI_MAG_PD", "MIMO_RI_PD", "MIMO_RI_PD_REF"]
idx = 1
all_est_dict = torch.load(os.path.join(log_dir, f'doa_1.0_{idx}_tol_5.0deg_euclid_True_wgt_mech_MASK.pt'))
for frm in range(250, 300, 5):
    plt.figure()
    for loss in loss_list:
        (est_f_doa, est_f_vals, est_utt_doa, est_frm_Acc, est_utt_Acc, metrics_list, est_mae_only_correct_frms, est_mae_only_incorrect_frms, est_mae_overall_only_vad_frms) = all_est_dict[f'{loss}']
        #plt.figure()
        plt.title(f'{frm}')
        plt.plot(est_f_vals.flatten('F')[361*(frm-1):361*frm]) #column (theta)
    plt.axvline(x=est_utt_doa)
    plt.axvline(x=360-est_utt_doa)
    
    #plt.plot(est_utt_doa*torch.ones(361))
    #plt.plot(360 - est_utt_doa*torch.ones(361))
    plt.legend(loss_list)

In [ ]:
from array_setup import *
array_type, num_mics, intermic_dist = "circular", 7, 4.25
array_setup = get_array_set_up_from_config(array_type, num_mics, intermic_dist)
local_mic_pos = torch.from_numpy(array_setup.mic_pos)
local_mic_center=torch.from_numpy(np.array([0.0, 0.0, 0.0]))
mic_pairs = [(mic_1, mic_2) for mic_1 in range(0, num_mics) for mic_2 in range(mic_1+1, num_mics)]

In [ ]:
from masked_gcc_phat import compute_vad

In [ ]:
#torch implementation
def gcc_phat_loc_orient(X, est_mask, fs, nfft, local_mic_pos, mic_center, src_mic_dist, weighted, sig_vad, is_euclidean_dist):

    (chan, frames, freq) = X.shape
    X_ph = torch.angle(X)
    X_ph_diff = X_ph[0,:,1:] - X_ph[1,:,1:]

    #weightage
    if weighted:
        est_mask_pq = torch.pow(est_mask[0,:,1:]*est_mask[1,:,1:], 1.0) # 0.3) # 0.5


    angular_freq = 2*torch.pi*fs*1.0/nfft*torch.arange(1, freq, dtype=torch.float32)

    angle_step = 1.0
    theta_grid = 360.0 if mic_center=="circular" else 180.0
    all_directions = torch.linspace(0,theta_grid,int(theta_grid/angle_step+1),dtype=torch.float32) #a set of potential directions
    dist = src_mic_dist
    c = 343

    all_directions_val = [] 
    delays = []
    all_info = []
    all_info_unweighted = []

    for ind, direction in enumerate(all_directions):
        #ang = (direction - 90)/180.0*torch.pi  #radians
        ang_doa = (direction)/180*torch.pi  #radians
        
        if is_euclidean_dist:
            src_pos = torch.tensor([torch.cos(ang_doa)*dist,torch.sin(ang_doa)*dist, 0.0],dtype=torch.float32) #+ mic_center
            dist_pp = torch.sqrt(torch.sum((src_pos-local_mic_pos[0])**2))  ## TODO: pp
            dist_qq = torch.sqrt(torch.sum((src_pos-local_mic_pos[1])**2))  ## TODO: qq
            delay = (dist_qq-dist_pp)/c #,device=est_mask.device)#.type_as(est_mask)   

        else:
            # ASSUMPTION on unit circle
            dist = 1
            src_pos = torch.tensor([torch.cos(ang_doa)*dist,torch.sin(ang_doa)*dist, 0.0],dtype=torch.float32) 
            delay = np.dot((local_mic_pos[0]-local_mic_pos[1]), src_pos)/c

            
        delays.append(delay)
        delay_vec = angular_freq*delay
        #print(X_ph_diff.shape, delay_vec.shape)
        gcc_phat_pq = torch.cos( X_ph_diff - delay_vec.to(device=X_ph_diff.device))

        all_info_unweighted.append(gcc_phat_pq)   #debug
        if weighted:
            mgcc_phat_pq = est_mask_pq*gcc_phat_pq
            gcc_phat_pq = mgcc_phat_pq

        all_info.append(gcc_phat_pq)              #debug
        per_direction_val = torch.sum(gcc_phat_pq, dim=1)

        all_directions_val.append(per_direction_val)

    vals = torch.stack(all_directions_val, dim=0)

    sig_vad_frms = sig_vad.shape[0]
    vals = vals[:,:sig_vad_frms]*sig_vad   ##caluation for only vad frames

    utt_sum = torch.sum(vals,dim=1)
    utt_doa_idx = torch.argmax(utt_sum)
    utt_doa = all_directions[utt_doa_idx]


    doa_idx = torch.argmax(vals,dim=0)
    doa = all_directions[doa_idx.to(all_directions.device)]

    return doa, vals, utt_doa, all_info_unweighted, all_info #utt_sum, delays

def gcc_phat_all_pairs(X: "[num_mics, T, F]", est_mask, fs, nfft, local_mic_pos, mic_center, src_mic_dist, weighted, sig_vad, is_euclidean_dist, mic_pairs):
    #print(X.shape)
    num_mics, num_frames, num_freq = X.shape
    #mic_pairs = [(mic_1, mic_2) for mic_1 in range(0, num_mics) for mic_2 in range(mic_1+1, num_mics)]
    theta_grid, mic_center = (361, "circular") if num_mics==7 else (181, "linear")
    pair_acc_X_frm_vals = torch.zeros(theta_grid, num_frames).to(X.device)  #theta_grid [0,180]

    #centre mics (2mic)
    #  Even mics
    idx = num_mics//2
    centre_mic_pair = (idx-1, idx) if num_mics!=7 else (1,4)  #'linear' in array_type 
    #print(centre_mic_pair)
   
    for mic_pair in mic_pairs:
        X_pair = X[[mic_pair[0], mic_pair[1]],:,:]
        est_mask_pair = est_mask[[mic_pair[0], mic_pair[1]],:,:]
        mic_pair_pos = local_mic_pos[[mic_pair[0], mic_pair[1]],:]

        X_pair_doa, X_frm_vals, X_pair_utt_doa, _, _ = gcc_phat_loc_orient(X_pair, est_mask_pair, fs, nfft, mic_pair_pos, 
															 mic_center, src_mic_dist, weighted, sig_vad, is_euclidean_dist)
        
        if centre_mic_pair == mic_pair:
            X_2mic_doa = X_pair_doa
            X_2mic_utt_doa = X_pair_utt_doa
            X_2mic_frm_vals = X_frm_vals

        pair_acc_X_frm_vals += X_frm_vals

    doa_idx = torch.argmax(pair_acc_X_frm_vals,dim=0)  #assuming doa_idx is doa

    utt_sum = torch.sum(pair_acc_X_frm_vals,dim=1)
    utt_doa_idx = torch.argmax(utt_sum)

    return doa_idx, pair_acc_X_frm_vals, utt_doa_idx, X_2mic_doa, X_2mic_utt_doa, X_2mic_frm_vals


In [ ]:
def get_mask_np( mix, est):
    noise = mix - est
    mask = np.sqrt( np.abs(est)**2/(np.abs(noise)**2 + np.abs(est)**2))
    return mask

In [ ]:
from multiprocessing import Process, Queue

In [ ]:
def gcc_phat_loc_orient_pair(X, est_mask, fs, nfft, local_mic_pos, mic_center, src_mic_dist, weighted, sig_vad, is_euclidean_dist, mic_pair, result_Q):

    (chan, frames, freq) = X.shape
    X_ph = np.angle(X)
    X_ph_diff = X_ph[0,:,1:] - X_ph[1,:,1:]

    #weightage
    if weighted:
        est_mask_pq = np.power(est_mask[0,:,1:]*est_mask[1,:,1:], 1.0) # 0.3) # 0.5


    angular_freq = 2*np.pi*fs*1.0/nfft*np.arange(1, freq, dtype=np.float32)

    angle_step = 1.0
    theta_grid = 360.0 if mic_center=="circular" else 180.0
    all_directions = np.linspace(0,theta_grid,int(theta_grid/angle_step+1),dtype=np.float32) #a set of potential directions
    dist = src_mic_dist
    c = 343

    all_directions_val = [] 
    delays = []
    all_info = []
    all_info_unweighted = []

    #print(f"Entering direction for loop ", flush=True)
    for ind, direction in enumerate(all_directions):
        #ang = (direction - 90)/180.0*torch.pi  #radians
        ang_doa = (direction)/180*np.pi  #radians
        
        if is_euclidean_dist:
            src_pos = np.array([np.cos(ang_doa)*dist,np.sin(ang_doa)*dist, 0.0],dtype=np.float32) #+ mic_center
            dist_pp = np.sqrt(np.sum((src_pos-local_mic_pos[0])**2))  ## TODO: pp
            dist_qq = np.sqrt(np.sum((src_pos-local_mic_pos[1])**2))  ## TODO: qq
            delay = (dist_qq-dist_pp)/c #,device=est_mask.device)#.type_as(est_mask)   

        else:
            # ASSUMPTION on unit circle
            dist = 1
            src_pos = np.array([np.cos(ang_doa)*dist,np.sin(ang_doa)*dist, 0.0],dtype=np.float32) 
            delay = np.dot((local_mic_pos[0]-local_mic_pos[1]), src_pos)/c

            
        delays.append(delay)
        delay_vec = angular_freq*delay
        #print(X_ph_diff.shape, delay_vec.shape)
        gcc_phat_pq = np.cos( X_ph_diff - delay_vec)#.to(device=X_ph_diff.device))

        all_info_unweighted.append(gcc_phat_pq)   #debug
        if weighted:
            mgcc_phat_pq = est_mask_pq*gcc_phat_pq
            gcc_phat_pq = mgcc_phat_pq

        all_info.append(gcc_phat_pq)              #debug
        per_direction_val = np.sum(gcc_phat_pq, axis=1)

        all_directions_val.append(per_direction_val)

    vals = np.stack(all_directions_val, axis=0)

    sig_vad_frms = sig_vad.shape[0]
    vals = vals[:,:sig_vad_frms]*sig_vad   ##caluation for only vad frames

    utt_sum = np.sum(vals,axis=1)
    utt_doa_idx = np.argmax(utt_sum)
    utt_doa = all_directions[utt_doa_idx]


    doa_idx = np.argmax(vals,axis=0)
    doa = all_directions[doa_idx]#.to(all_directions.device)]

    doa_dict = { "mic_pair": mic_pair, "doa": doa, "vals": vals, "utt_doa": utt_doa} #, "all_info_unweighted": all_info_unweighted, "all_info": all_info}
    #print(f"Generated result Dict GCC-PHAT pair ", flush=True)
    result_Q.put(doa_dict)
    return  #doa, vals, utt_doa, all_info_unweighted, all_info #utt_sum, delays

In [ ]:
def gcc_phat_all_pairs_parallel(X: "[num_mics, T, F]", est_mask, fs, nfft, local_mic_pos, mic_center, src_mic_dist, weighted, sig_vad, is_euclidean_dist, mic_pairs):
    #print(X.shape)
    num_mics, num_frames, num_freq = X.shape
    #mic_pairs = [(mic_1, mic_2) for mic_1 in range(0, num_mics) for mic_2 in range(mic_1+1, num_mics)]
    theta_grid, mic_center = (361, "circular") if num_mics==7 else (181, "linear")

    pair_acc_X_frm_vals = np.zeros((theta_grid, num_frames))#.to(X.device)  #theta_grid [0,180]

    #centre mics (2mic)
    #  Even mics
    idx = num_mics//2
    centre_mic_pair = (idx-1, idx) if num_mics!=7 else (1,4)  #'linear' in array_type 
    #print(centre_mic_pair)


    result_Q = Queue()
    results = []
    result_pair_list = []

    for mic_pair in mic_pairs:
        X_pair = X[[mic_pair[0], mic_pair[1]],:,:]
        est_mask_pair = est_mask[[mic_pair[0], mic_pair[1]],:,:]
        mic_pair_pos = local_mic_pos[[mic_pair[0], mic_pair[1]],:]

        #print(f"Launching GCC-PHAT pair : {mic_pair}", flush=True)
        p = Process(target=gcc_phat_loc_orient_pair, args=(X_pair, est_mask_pair, fs, nfft, mic_pair_pos, 
													 mic_center, src_mic_dist, weighted, sig_vad, is_euclidean_dist, mic_pair, result_Q))
        p.start()			
        results.append(p)

    for result in results:
        est_dict = result_Q.get()
        result_pair_list.append(est_dict)
        pair_acc_X_frm_vals += est_dict["vals"] #X_frm_vals

    		# wait for all processes to complete
    for result in results:
        result.join()


    doa_idx = np.argmax(pair_acc_X_frm_vals,axis=0)  #assuming doa_idx is doa

    utt_sum = np.sum(pair_acc_X_frm_vals,axis=1)
    utt_doa_idx = np.argmax(utt_sum)

    return doa_idx, pair_acc_X_frm_vals, utt_doa_idx, result_pair_list #, X_2mic_doa, X_2mic_utt_doa, X_2mic_frm_vals


In [ ]:
def get_acc(est_vad, est_blk_val, tgt_blk_val, tol=5, vad_th=0.6):
    n_blocks = len(est_vad)
    non_vad_blks =[]
    acc=0.0
    valid_blk_count = 0.0
    mae_only_correct_frms, mae_only_incorrect_frms, mae_overall_only_vad_frms = 0.0,0.0,0.0
    for idx in range(0, n_blocks):
        if est_vad[idx] >=vad_th:
            diff = np.abs(est_blk_val[idx] - tgt_blk_val[idx]) 
            diff = 360-diff if diff > 180 else diff
            if ( diff <= tol):
                acc += 1.0
                mae_only_correct_frms += diff
            else:
                mae_only_incorrect_frms += diff
            
            mae_overall_only_vad_frms += diff
            valid_blk_count += 1.0
        else:
            non_vad_blks.append(idx)
    if acc !=0:
        mae_only_correct_frms /= acc
    if (valid_blk_count-acc) !=0:
        mae_only_incorrect_frms /= (valid_blk_count-acc)

    mae_overall_only_vad_frms /= valid_blk_count

    acc /= valid_blk_count
    
    #print(f'n_blocks: {n_blocks}, non_vad_blks: {non_vad_blks}')
    return acc, mae_only_correct_frms, mae_only_incorrect_frms, mae_overall_only_vad_frms

In [ ]:
from metrics import *

In [ ]:
#debugging 
#circular array analysis
log_dir = os.path.join(dbg_log_dir, case_log_dir)

loss_list = ["MIMO_RI", "MIMO_RI_MAG", "MIMO_RI_MAG_PD", "MIMO_RI_PD", "MIMO_RI_PD_REF"]
idx, wgt_mech = 7, "MASK"
nfft=320
fs, c, freq_bins = 16000, 343, np.arange(1, nfft//2) 

#****
#mic_pairs = [(0, mic_2) for mic_2 in range(1, num_mics)]
#mic_pairs.append((1,4))
#mic_pairs.append((2,5))
#mic_pairs.append((3,6))
mic_pairs = [(1,4)]

# mix tgt signals
mix_sig, fs = torchaudio.load(os.path.join(log_dir, f'mix_{idx}.wav'))
tgt_sig, fs = torchaudio.load(os.path.join(log_dir, f'tgt_{idx}.wav'))

mix_metrics = eval_metrics_batch_v1(tgt_sig[[0],:].numpy(), mix_sig[[0],:].numpy())

mix_X = torch.stft(mix_sig, nfft, nfft // 2, nfft, torch.hamming_window(nfft), return_complex=True, center = False)
#X = X[:,1:,:] # removing the dc component
mix_X = torch.permute(mix_X, [0,2,1])

tgt_X = torch.stft(tgt_sig, nfft, nfft // 2, nfft, torch.hamming_window(nfft), return_complex=True, center = False)
#X = X[:,1:,:] # removing the dc component
tgt_X = torch.permute(tgt_X, [0,2,1])


tgt_vad = compute_vad(tgt_sig[0,:].numpy(), nfft, nfft//2)
src_mic_dist, weighted, sig_vad, is_euclidean_dist = 1, True, tgt_vad, True

mix_mask = get_mask_np(mix_X, mix_X)

if wgt_mech=="MASK":
    tgt_mask = get_mask_np(mix_X, tgt_X)
elif wgt_mech=="MAG":
    tgt_mask = torch.sqrt(torch.abs(tgt_X))
else:
    tgt_mask = get_mask_np(mix_X, mix_X)

mix_doa_idx, mix_pair_acc_X_frm_vals, mix_utt_doa_idx, mix_result_pair_list = gcc_phat_all_pairs_parallel(mix_X.numpy(), mix_mask.numpy(), fs, nfft, local_mic_pos.numpy(), local_mic_center, src_mic_dist, weighted, sig_vad.numpy(), is_euclidean_dist, mic_pairs)
tgt_doa_idx, tgt_pair_acc_X_frm_vals, tgt_utt_doa_idx, tgt_result_pair_list = gcc_phat_all_pairs_parallel(tgt_X.numpy(), tgt_mask.numpy(), fs, nfft, local_mic_pos.numpy(), local_mic_center, src_mic_dist, weighted, sig_vad.numpy(), is_euclidean_dist, mic_pairs)


print(f"mix, acc: {get_acc(tgt_vad, mix_doa_idx, tgt_doa_idx, tol=5)}")

results_loss_list = []
for loss_idx, loss in enumerate(loss_list):
    sig, fs = torchaudio.load(os.path.join(log_dir, f'{idx}_{loss_idx}_{loss}.wav'))

    est_metrics = eval_metrics_batch_v1(tgt_sig[[0],:].numpy(), sig[[0],:].numpy())

    X = torch.stft(sig, nfft, nfft // 2, nfft, torch.hamming_window(nfft), return_complex=True, center = False)
    #X = X[:,1:,:] # removing the dc component
    X = torch.permute(X, [0,2,1])

    if wgt_mech=="MASK":
        est_mask = get_mask_np(mix_X, X)
    elif wgt_mech=="MAG":
        est_mask = torch.sqrt(torch.abs(X))
    else:
        mix_mask = get_mask_np(mix_X, mix_X)

    print("Launching GCC-PHAT")
    #doa_idx, pair_acc_X_frm_vals, utt_doa_idx, X_2mic_doa, X_2mic_utt_doa, X_2mic_frm_vals = gcc_phat_all_pairs(X, est_mask, fs, nfft, local_mic_pos, local_mic_center, src_mic_dist, weighted, sig_vad, is_euclidean_dist, mic_pairs)
    #print("Non parallel version")
    doa_idx, pair_acc_X_frm_vals, utt_doa_idx, result_pair_list = gcc_phat_all_pairs_parallel(X.numpy(), est_mask.numpy(), fs, nfft, local_mic_pos.numpy(), local_mic_center, src_mic_dist, weighted, sig_vad.numpy(), is_euclidean_dist, mic_pairs)

    results_loss_list.append(result_pair_list)

    print(f"{loss}, acc: {get_acc(tgt_vad, doa_idx, tgt_doa_idx)}, dsnr: {est_metrics['snr']-mix_metrics['snr']}, dstoi: {est_metrics['stoi']-mix_metrics['stoi']}, dpesq: {est_metrics['pesq_nb']-mix_metrics['pesq_nb']})")

In [ ]:
print(get_acc(tgt_vad, mix_doa_idx, tgt_doa_idx, tol=10))
print(get_acc(tgt_vad, doa_idx, tgt_doa_idx, tol=10))

print(get_acc(tgt_vad, mix_doa_idx, tgt_doa_idx, tol=5))
print(get_acc(tgt_vad, doa_idx, tgt_doa_idx, tol=5))

In [ ]:
tgt_utt_doa_idx, utt_doa_idx, doa_idx, tgt_doa_idx 

In [ ]:
frm = 50

for _pair in mix_result_pair_list:
    mix_f_doa_pair = _pair["doa"]
    mix_f_vals_pair = _pair["vals"]
    plt.figure()
    plt.title(f'{_pair["mic_pair"]}')
    plt.plot(mix_f_vals_pair.flatten('F')[361*(frm-1):361*frm])
    plt.plot(mix_pair_acc_X_frm_vals.flatten('F')[361*(frm-1):361*frm]/len(mic_pairs), 'r')
    plt.axvline(x=tgt_utt_doa_idx, c='r')
    plt.axvline(x=(360-tgt_utt_doa_idx)%360, c='g')
    plt.axvline(x=(120-tgt_utt_doa_idx)%360, c='b')
    plt.axvline(x=(240-tgt_utt_doa_idx)%360, c='y')

In [ ]:
frm = 2

for _pair in tgt_result_pair_list:
    tgt_f_doa_pair = _pair["doa"]
    tgt_f_vals_pair = _pair["vals"]
    plt.figure()
    plt.title(f'{_pair["mic_pair"]}')
    plt.plot(tgt_f_vals_pair.flatten('F')[361*(frm-1):361*frm])
    plt.plot(tgt_pair_acc_X_frm_vals.flatten('F')[361*(frm-1):361*frm]/len(mic_pairs), 'r')
    plt.axvline(x=tgt_utt_doa_idx, c='r')
    plt.axvline(x=(360-tgt_utt_doa_idx)%360, c='g')
    plt.axvline(x=(120-tgt_utt_doa_idx)%360, c='b')
    plt.axvline(x=(240-tgt_utt_doa_idx)%360, c='y')
    #plt.plot(est_f_doa_pair)

In [ ]:
np.argmax(pair_acc_X_frm_vals.flatten('F')[361*(frm-1):361*frm])

In [ ]:
frm = 2
for _pair in result_pair_list:
    est_f_doa_pair = _pair["doa"]
    est_f_vals_pair = _pair["vals"]
    plt.figure()
    plt.title(f'{_pair["mic_pair"]}')
    plt.plot(est_f_vals_pair.flatten('F')[361*(frm-1):361*frm])
    plt.plot(pair_acc_X_frm_vals.flatten('F')[361*(frm-1):361*frm]/len(mic_pairs), 'r')
    plt.axvline(x=tgt_utt_doa_idx, c='r')
    plt.axvline(x=(360-tgt_utt_doa_idx)%360, c='g')
    plt.axvline(x=(120-tgt_utt_doa_idx)%360, c='b')
    plt.axvline(x=(240-tgt_utt_doa_idx)%360, c='y')
    #plt.plot(est_f_doa_pair)

In [ ]:
print(get_acc(tgt_vad, doa_idx, tgt_doa_idx))

In [ ]:
frm = 20
for loss_idx, result_pair_list in enumerate(results_loss_list):
    for _pair in result_pair_list:
        est_f_doa_pair = _pair["doa"]
        est_f_vals_pair = _pair["vals"]
        plt.figure()
        plt.title(f'{loss_list[loss_idx]}_{_pair["mic_pair"]}')
        plt.plot(est_f_vals_pair.flatten('F')[361*(frm-1):361*frm])
        plt.plot(pair_acc_X_frm_vals.flatten('F')[361*(frm-1):361*frm]/len(mic_pairs), 'r')
        plt.axvline(x=tgt_utt_doa_idx, c='r')
        plt.axvline(x=(360-tgt_utt_doa_idx)%360, c='g')
        plt.axvline(x=(120-tgt_utt_doa_idx)%360, c='b')
        plt.axvline(x=(240-tgt_utt_doa_idx)%360, c='y')

In [ ]:
loss_idx = 4
frm = 51
for _pair in results_loss_list[loss_idx]:
    est_f_doa_pair = _pair["doa"]
    est_f_vals_pair = _pair["vals"]
    plt.figure()
    plt.title(f'{_pair["mic_pair"]}')
    plt.plot(est_f_vals_pair.flatten('F')[361*(frm-1):361*frm])
   # plt.plot(pair_acc_X_frm_vals.flatten('F')[361*(frm-1):361*frm]/len(mic_pairs), 'r')
    plt.axvline(x=tgt_utt_doa_idx, c='r')
    plt.axvline(x=(360-tgt_utt_doa_idx)%360, c='g')
    plt.axvline(x=(120-tgt_utt_doa_idx)%360, c='b')
    plt.axvline(x=(240-tgt_utt_doa_idx)%360, c='y')

In [ ]:

for frm in range(0, 399, 10):
    plt.figure()
    plt.title(f'frm_{frm}_{_pair["mic_pair"]}')
    for loss_idx in range(0,4):
        for _pair in results_loss_list[loss_idx]:
            est_f_doa_pair = _pair["doa"]
            est_f_vals_pair = _pair["vals"]
            plt.plot(est_f_vals_pair.flatten('F')[361*(frm-1):361*frm])
        # plt.plot(pair_acc_X_frm_vals.flatten('F')[361*(frm-1):361*frm]/len(mic_pairs), 'r')
    plt.axvline(x=tgt_utt_doa_idx, c='r')
    plt.axvline(x=(360-tgt_utt_doa_idx)%360, c='g')
    plt.axvline(x=(120-tgt_utt_doa_idx)%360, c='b')
    plt.axvline(x=(240-tgt_utt_doa_idx)%360, c='y')